# IOT-IPS

The script for iot-ips project

Current use a simple fully-connected classifer with averaged rssi + csi information.

In [1]:
import os
import sys
import csv
import math
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

/home/rtu/anaconda3/envs/habitat/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Data

## Define the input format

In [3]:
def parser(rssi, csi, position):
    '''
    description:
        this is a function to rearrange the raw data into the form of your model input
        so that you can easily design the input format and arrange the input data.
        
    input:
        rssi: the averaged rssi in one collect file, stored in numpy array
        csi: an array consists of all csi data in one collect file, stored in a 2D numpy array
        position: nparray(x,y)
        Note: both the rssi and csi belong to the same location
        
    output:
        x: the list containing inputs of the model
        y: the correspond ground truth
    '''
    
    signal = np.concatenate((rssi,csi.mean(axis=0)))
    y = np.array(position, dtype=np.int32)
    #onehot = np.zeros((27,), dtype=int)
    #onehot[int(position[0]+position[1]*3)]=1
    
    x = [signal,]
    y = [y,]
    
    x = [torch.Tensor(i) for i in x]
    y = [torch.LongTensor(i) for i in y]
    #print(y)
        
    return x, y # use rssi data only as example

## Dataset

In [4]:
class Dataset(Dataset):
    
    def __init__(self, path, parser):
        filenames = os.listdir(path)
        self.parser = parser
        
        x_ = []
        y_ = []
        
        for name in filenames:
            if 'csi' in name:
                file = os.path.join(path, name)
                x,y = self.readOne(file)
                x_.extend(x)
                y_.extend(y)
        
        self.x = x_
        self.y = y_
                      
    def readOne(self, fname):
        
        if 'csi' in fname:
            csi = fname
            rssi = fname.replace('csi','rssi')
        elif 'rssi' in fname:
            rssi = fname
            csi = fname.replace('rssi','csi')
        
        # parse the position from the file name
        position = csi[csi.find('csi')+3: csi.rfind('.')].split('_')
        position = np.asfarray([float(i) for i in position])
        
        rssi_f = open(rssi)
        csi_f = open(csi)
        rssilines = rssi_f.readlines()
        csilines = csi_f.readlines()
        
        # Read the rssi and average
        row = 0
        rssi_data = np.zeros(8, dtype=float)
        for line in rssilines:
            rssi_raw = line.split(',')
            try:
                rssi_data += np.asfarray([int(i) for i in rssi_raw])
                row += 1
            except:
                continue
        rssi_data /= row
            
        # Read the csi
        csi_data = []
        for line in csilines:
            csi_raw = line.split(',')[-2][1:-2]  # skip the brackets and space
            csi_str = csi_raw.split(' ')
            csi_float = np.asfarray([int(i) for i in csi_str])
            csi_data.append(csi_float)
        csi_data = np.asfarray(csi_data)
        
        rssi_f.close()
        csi_f.close()
        
        x,y = self.parser(rssi_data, csi_data, position)
        
        return x,y
        
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Model

In [26]:
class Model(nn.Module):
    def __init__(self, input_dim=136, hidden=32):
        super(Model, self).__init__()
        
        layers = [
            nn.Linear(input_dim,hidden),
            nn.LeakyReLU(),
            nn.Linear(hidden,27),
        ]
        
        self.xlayers = nn.Sequential(*layers)
        
    def forward(self, x):
        x = self.xlayers(x)
        return x

# Hyper Parameters

In [33]:
lr = 0.0005
momentum = 0.03
n_epochs = 10000
batch_size = 32
valid_epoch = 1000
save_path = './test.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [34]:
train_dataset = Dataset('data/train', parser)
test_dataset = Dataset('data/test', parser)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
valid_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

model = Model().to(device)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum) 

best_loss, step, early_stop_count = math.inf, 0, 0

train_pbar = tqdm(range(n_epochs), position=0, leave=True)
for epoch in train_pbar:
    
    model.train()
    
    acc_record = []
    loss_record = []

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x+torch.rand(x.size()).to(device))  
        
        y = (y[:,0]+y[:,1]*3)
        loss = criterion(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        step += 1
        
        _,pred = torch.max(pred, 1)
        acc = (pred==y).sum().item()
        acc_record.append(acc)
        loss_record.append(loss.detach().item())

        train_pbar.set_postfix({'loss': loss.detach().item()})
    train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
    
    mean_train_loss = sum(loss_record)/len(loss_record)
    mean_train_acc = sum(acc_record)/len(train_dataset)

    if epoch % valid_epoch==0 or epoch==n_epochs-1:

        acc_record = []
        
        model.eval() # Set your model to evaluation mode.
        loss_record = []
        with torch.no_grad():
            for x, y in valid_loader:
                x, y = x.to(device), y.to(device)
                pred = model(x)
                
                y = (y[:,0]+y[:,1]*3)
                _,pred = torch.max(pred, 1)
                acc = (pred==y).sum().item()
                acc_record.append(acc)

        mean_valid_acc = sum(acc_record)/len(test_dataset)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Train acc: {mean_train_acc:.4f}, Valid acc: {mean_valid_acc:.4f}')
    
        if mean_train_loss < best_loss:
            best_loss = mean_train_loss
            torch.save(model.state_dict(), save_path)
            print('Saving model with loss {:.3f}...'.format(best_loss))


Epoch [51/10000]:   0%|          | 50/10000 [00:00<00:40, 247.79it/s, loss=3.32]

Epoch [1/10000]: Train loss: 6.1288, Train acc: 0.0000, Valid acc: 0.0000
Saving model with loss 6.129...


Epoch [1058/10000]:  11%|▋     | 1056/10000 [00:03<00:31, 282.73it/s, loss=2.53]

Epoch [1001/10000]: Train loss: 2.5689, Train acc: 0.2963, Valid acc: 0.0000
Saving model with loss 2.569...


Epoch [2060/10000]:  20%|█▏    | 2048/10000 [00:07<00:27, 291.59it/s, loss=1.85]

...

Epoch [8001/10000]: Train loss: 0.2371, Train acc: 1.0000, Valid acc: 0.0000
Saving model with loss 0.237...


Epoch [9063/10000]:  91%|█████▍| 9052/10000 [00:31<00:03, 307.94it/s, loss=0.19]

Epoch [9001/10000]: Train loss: 0.1886, Train acc: 1.0000, Valid acc: 0.0000
Saving model with loss 0.189...


Epoch [10000/10000]: 100%|███| 10000/10000 [00:34<00:00, 285.93it/s, loss=0.146]

Epoch [10000/10000]: Train loss: 0.1463, Train acc: 1.0000, Valid acc: 0.0000
Saving model with loss 0.146...


In [39]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)

model.eval() # Set your model to evaluation mode.
loss_record = []
val_acc = 0
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        
        pred = model(x)
        _, pred = torch.max(pred, 1) 
        pred = pred.cpu().detach().numpy()[0]
        
        print(f'Prediction: {np.asarray([pred%3, pred//3])}')
        print(f'GT        : {y.cpu().detach().numpy()[0]}\n\n')

Prediction: [0 6]
GT        : [1 7]


Prediction: [0 6]
GT        : [-1  6]


Prediction: [1 6]
GT        : [4 3]


Prediction: [0 6]
GT        : [1 9]


Prediction: [0 6]
GT        : [1 5]


Prediction: [1 4]
GT        : [-1  9]


